In [1]:
from grobid_client.grobid_client import GrobidClient
import json
import xmltodict
import nltk
from nltk import tokenize
from nltk.tokenize import word_tokenize
import grobid_tei_xml
import re
from difflib import SequenceMatcher
import spacy

nlp = spacy.load("en_core_web_sm")

In [2]:
# client = GrobidClient(config_path="/Users/revglue/grobid_client_python/config.json")
# client.process("processFulltextDocument", "/Users/revglue/study/main_work/my_working/testing_pdf", output="/Users/revglue/study/main_work/my_working/test_out", consolidate_citations=True, tei_coordinates=True,segment_sentences=True, n=20, force=True)

In [3]:
def search_in_sentence(strs, word,heading):
    ratio_list = []
    for sentence in strs:
        random_sen=""
        if(sentence.startswith(heading)):
            random_sen=sentence.replace(heading,"").strip()
        else:
            random_sen=sentence.strip()
        ratio = SequenceMatcher(None, random_sen, word).ratio()
        ratio_tuple=[random_sen,ratio]
        ratio_list.append(ratio_tuple)
#         if(ratio > 0.8):
#             print(ratio)
#             return random_sen
    max_val=max(ratio_list, key=lambda x: x[1])
    return max_val

In [4]:
def find_between(s, first, last):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [5]:
def check_target_exist(array):
    for val in array:
        if '@target' in val:
            return True
    return False    

In [6]:
def check_bibr_exist(array):
    for index,val in enumerate(array):
        if "@type" in val: 
#             print(str(index+1)+": "+str(val))
            if val["@type"] == "bibr":
                return True
        else:
            return False
    return False    

In [7]:
def extract_citation(sentence):            
    citing_sentences = []
    citing_sentences_raw = []


    if 's' in sentence["p"]:
        if 'ref' in sentence["p"]["s"]:
            if '@type' in sentence["p"]["s"]["ref"]:
                if sentence["p"]["s"]["ref"]["@type"] == "bibr" and '@target' in sentence["p"]["s"]["ref"]:
                    citing_sentences_raw.append(sentence["p"]["s"]["#text"])
            else:
                if check_bibr_exist(sentence["p"]["s"]["ref"]) and check_target_exist(sentence["p"]["s"]["ref"]):
                    citing_sentences_raw.append(sentence["p"]["s"]["#text"])
        else:
            for inner_citing in sentence["p"]["s"]:
                if 'ref' in inner_citing:
                    if '@type' in inner_citing["ref"]:
                        if inner_citing["ref"]["@type"] == "bibr" and '@target' in inner_citing["ref"]:
                            citing_sentences_raw.append(inner_citing["#text"])
                    else:
                        if check_bibr_exist(inner_citing["ref"]) and check_target_exist(inner_citing["ref"]):
                            citing_sentences_raw.append(inner_citing["#text"])
    else:
        for citing in sentence["p"]:
            if 'ref' in citing["s"]:
                if '@type' in citing["s"]["ref"]:
                    if citing["s"]["ref"]["@type"] == "bibr" and '@target' in citing["s"]["ref"]:
                        citing_sentences_raw.append(citing["s"]["#text"])
                else:
                    if check_bibr_exist(citing["s"]["ref"]) and check_target_exist(citing["s"]["ref"]):
                        citing_sentences_raw.append(citing["s"]["#text"])
            else:
                for inner_citing in citing["s"]:
                    if 'ref' in inner_citing:
                        if '@type' in inner_citing["ref"]:
                            if inner_citing["ref"]["@type"] == "bibr" and '@target' in inner_citing["ref"]:
                                citing_sentences_raw.append(inner_citing["#text"])
                        else:
                            if check_bibr_exist(inner_citing["ref"]) and check_target_exist(inner_citing["ref"]):
                                citing_sentences_raw.append(inner_citing["#text"])
    for sentence in citing_sentences_raw:
        citing_sentences.append(search_in_sentence(paper_sentence_list,sentence,heading)[0])        
    body.append([heading,citing_sentences,len(citing_sentences)])

In [8]:
xml_path = "/Users/revglue/study/main_work/my_working/test_out/E12-1072.tei.xml"

with open(xml_path, 'r') as xml_file:   
    doc = grobid_tei_xml.parse_document_xml(xml_file.read())

json_body = json.loads(json.dumps(doc.to_dict(), indent=2))
paper_text = json_body["body"]
paper_text=paper_text.replace("et al. ","et al., ")
paper_sentence_list = tokenize.sent_tokenize(paper_text)
# print(paper_sentence_list)




xml=xmltodict.parse(open(xml_path).read())
json_text = json.loads(json.dumps(xml))
print(json_text)
print("\n")
headings=[]
body = []

for sentence in json_text["TEI"]["text"]["body"]["div"]:
    if "#text" in sentence["head"]:
        heading = sentence["head"]["#text"]
    else:
        heading = sentence["head"]
        if heading.isnumeric():
            if 's' in sentence["p"]:
                if '#text' in sentence["p"]["s"]:
                    heading=sentence["p"]["s"]["#text"]
                else:
                    heading=sentence["p"]["s"][0]["#text"]
            else:
                if '#text' in sentence["p"][0]["s"]:
                    heading=sentence["p"][0]["s"]["#text"]
                else:
                    heading=sentence["p"][0]["s"][0]["#text"]
        else:
            heading = sentence["head"]
            
    extract_citation(sentence)

for item in body:
    print(item[0]+":"+"\n")
    for index,citings in enumerate(item[1]):
        print("\t"+str(index+1)+". "+citings)
    print("Total Citations: "+str(item[2])+"\n")
#     author = "(?:[A-Z][A-Za-z'`-]+)"
#     etal = "(?:et al.?)"
#     additional = "(?:,? (?:(?:and |& )?" + author + "|" + etal + "))"
#     year_num = "(?:19|20)[0-9][0-9]"
#     page_num = "(?:, p.? [0-9]+)?"  # Always optional
#     year = "(?:, *"+year_num+page_num+"| *\("+year_num+page_num+"\))"
#     regex = "(" + author + additional+"*" + year + ")"
#     citing = []
#     doc = nlp(item)
# #     print(headings[index])
# #     print(body[index]+"\n")
#     l = [sent.text for sent in doc.sents]
#     for sentence in l:
# #         if re.findall(r'\(([^)]+)?(?:19|20)\d{2}?([^)]+)?\)', sentence):
#         if re.findall(regex, sentence):
#             citing.append(sentence)
# #             print(sentence+"\n")
# #     print("Citing Sentences: "+str(len(citing))+"\n")

{'TEI': {'@xml:space': 'preserve', '@xmlns': 'http://www.tei-c.org/ns/1.0', '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance', '@xsi:schemaLocation': 'http://www.tei-c.org/ns/1.0 https://raw.githubusercontent.com/kermitt2/grobid/master/grobid-home/schemas/xsd/Grobid.xsd', '@xmlns:xlink': 'http://www.w3.org/1999/xlink', 'teiHeader': {'@xml:lang': 'en', 'fileDesc': {'titleStmt': {'title': {'@level': 'a', '@type': 'main', '#text': 'Elliphant: Improved Automatic Detection of Zero Subjects and Impersonal Constructions in Spanish'}}, 'publicationStmt': {'publisher': None, 'availability': {'@status': 'unknown', 'licence': None}}, 'sourceDesc': {'biblStruct': {'analytic': {'author': [{'persName': {'@coords': '1,123.00,142.40,49.48,10.75;1,172.48,140.87,1.41,6.99', 'forename': {'@type': 'first', '#text': 'Luz'}, 'surname': 'Rello'}}, {'persName': {'@coords': '1,242.23,142.40,106.22,10.75', 'forename': {'@type': 'first', '#text': 'Ricardo'}, 'surname': 'Baeza-Yates'}}, {'persName': {'@co

Introduction:

	1. Various natural language processing (NLP) tasks benefit from the identification of elliptical subjects, primarily anaphora resolution  (Mitkov, 2002)  and co-reference resolution  (Ng and Cardie, 2002) .
	2. The difficulty in detecting missing subjects and non-referential pronouns has been acknowledged since the first studies on the computational treatment of anaphora  (Hobbs, 1977; Hirst, 1981) .
	3. However, this task is of crucial importance when processing pro-drop languages since subject ellipsis is a pervasive phenomenon in these languages  (Chomsky, 1981) .
	4. The necessity of identifying such kind of elliptical constructions has been specifically highlighted in work about Spanish zero pronouns  (Ferrández and Peral, 2000)  and co-reference resolution  (Recasens and Hovy, 2009) .
	5. This work is an extension of the first author master's thesis  (Rello, 2010)  and a preliminary version of the algorithm was presented in  Rello et al., (2010) .
Total Citations: